<a href="https://colab.research.google.com/github/alegomezri/Image-based-EEG-Representation/blob/main/Entrenamiento_WDCNN_LAPLACIAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases  #Package for database reading.
!pip install mne #The MNE Package is installed
FILEID = "1-bPsREsUCOiJHzIqi8DQrfSjTAf5VAW_"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.
!dir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/UN-GCPDS/python-gcpds.databases to /tmp/pip-req-build-nodtc680
  Running command git clone --filter=blob:none --quiet https://github.com/UN-GCPDS/python-gcpds.databases /tmp/pip-req-build-nodtc680
  Resolved https://github.com/UN-GCPDS/python-gcpds.databases to commit c35637e1a19d7cd21656496339c1dedae6714916
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 38.1 MB/s eta 0:00:00
  Created wheel for gcpds-databases: filename=gcpds_databases-0.2-py3-none-any.whl size=94506 sha256=7f198a7a9270d919be009fa403ed027467a7bc7f9afd3ee5581ec595a09cdae9
  Stored in directory: /tmp/pip-ephem-wheel-cache-fa6nits1/wheels/8e/66/a7/91b78b1787a3e4d17cb82ea2da67845aa9389012c0ed8280b0
Successfully built gcpds-databases
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-05-17 00:

In [ ]:
from google.colab.drive import mount
from gcpds.databases import GIGA_MI_ME
import numpy as np
import mne
from MI_EEG_ClassMeth.utils import min_max_normalization
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr, Power_based_Connectivities, Topoplot
from mne.channels import make_standard_montage
from mne import create_info
from time import time
import matplotlib
import matplotlib.pyplot as plt
from mne.viz import plot_topomap
import os
import scipy.io as sio
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV,train_test_split
from sklearn.metrics import make_scorer, cohen_kappa_score
from pickle import load, dump
from MI_EEG_ClassMeth.Classifiers import WDCNN
from MI_EEG_ClassMeth.Preprocessing import spherical_spline_surface_laplacian

In [ ]:
mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
root_dir = "/content/drive/Shareddrives/GFC + WDCNN/Pruebas WDCNN2"

In [ ]:
parent_dir = os.path.join(root_dir,'Results_Prueba_laplacian_Todos/')
model_dir = os.path.join(parent_dir,'Models/')
grid_search_results_dir = os.path.join(parent_dir,'grid_search_results/')
# os.makedirs(model_dir)
# os.makedirs(grid_search_results_dir)

In [ ]:
grid_search_results_dir

'/content/drive/Shareddrives/GFC + WDCNN/Pruebas WDCNN2/Results_Prueba_laplacian_Todos/grid_search_results/'

# 5 Folds 80%, 20%

In [ ]:
#EXPERIMENTO 1
# GK + WDCNN
subjects = np.arange(52)+1
subjects = np.delete(subjects,[28,33])
for sbj in range(25,53):
    X_gk  = np.load('/content/drive/Shareddrives/GFC + WDCNN/Pruebas WDCNN2/Topograms_Prueba_laplacian_Todos/Gaussian_Kernel/sbj'+str(sbj)+'.npy')
    X_gk = np.moveaxis(X_gk, -1, 4).reshape(X_gk.shape[0], X_gk.shape[1], X_gk.shape[2], X_gk.shape[3], -1)
    y  = np.load('/content/drive/Shareddrives/GFC + WDCNN/Pruebas WDCNN2/Topograms_Prueba_laplacian_Todos/Targets/sbj'+str(sbj)+'.npy')
    estimator = WDCNN(output_units=np.unique(y).shape[0])
    hyparams = {'validation_split':[0, 0.1],
                'l1_l2':[1e-3, 1e-2, 1e-1],
                'hidden_units':[400, 800, 1200]
              }
    scores = {'acc': 'accuracy','kappa': make_scorer(cohen_kappa_score,greater_is_better=True)}
    Xtrain, Xtest, ytrain,ytest = train_test_split(X_gk,y,test_size=0.2,random_state=0)
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    grid_search = GridSearchCV(estimator, hyparams, scoring=scores, refit='acc',  cv=cv, verbose=2,
                          error_score='raise', return_train_score=True)
    print('Sujeto: ',sbj)
    grid_search.fit(Xtrain, ytrain)

    dump(grid_search.cv_results_, open(grid_search_results_dir + 'subject_'+str(sbj)+'.txt', 'wb'))
    grid_search.best_estimator_.get_model(model_dir + 'subject_'+str(sbj)+'.h5')

Sujeto:  25
Fitting 5 folds for each of 18 candidates, totalling 90 fits
4/4 [==============================] - 0s 38ms/step
[CV] END ..hidden_units=400, l1_l2=0.001, validation_split=0; total time=  22.7s
4/4 [==============================] - 0s 3ms/step
[CV] END ..hidden_units=400, l1_l2=0.001, validation_split=0; total time=   9.6s


1/1 [==============================] - 0s 221ms/step


4/4 [==============================] - 0s 3ms/step
[CV] END ..hidden_units=400, l1_l2=0.001, validation_split=0; total time=   9.6s
4/4 [==============================] - 1s 4ms/step
[CV] END ..hidden_units=400, l1_l2=0.001, validation_split=0; total time=   9.7s
4/4 [==============================] - 0s 3ms/step
[CV] END ..hidden_units=400, l1_l2=0.001, validation_split=0; total time=   9.8s
4/4 [==============================] - 0s 4ms/step
[CV] END hidden_units=400, l1_l2=0.001, validation_split=0.1; total time=  16.4s
4/4 [==============================] - 0s 3ms/step
[CV] END hidden_units=400, l1_l2=0.001, validation_split=0.1; total time=  16.1s
4/4 [==============================] - 0s 3ms/step
[CV] END hidden_units=400, l1_l2=0.001, validation_split=0.1; total time=  16.8s
4/4 [==============================] - 0s 3ms/step
[CV] END hidden_units=400, l1_l2=0.001, validation_split=0.1; total time=  16.1s
4/4 [==============================] - 0s 4ms/step
[CV] END hidden_units=400

FileNotFoundError: ignored